In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [13]:
data = pd.read_csv("/content/creditcard.csv")
print(data.head())

   Time        V1        V2        V3  ...       V27       V28  Amount  Class
0   0.0 -1.359807 -0.072781  2.536347  ...  0.133558 -0.021053  149.62    0.0
1   0.0  1.191857  0.266151  0.166480  ... -0.008983  0.014724    2.69    0.0
2   1.0 -1.358354 -1.340163  1.773209  ... -0.055353 -0.059752  378.66    0.0
3   1.0 -0.966272 -0.185226  1.792993  ...  0.062723  0.061458  123.50    0.0
4   2.0 -1.158233  0.877737  1.548718  ...  0.219422  0.215153   69.99    0.0

[5 rows x 31 columns]


In [14]:
data.shape

(179610, 31)

In [15]:
data = data.dropna()

In [16]:
data.shape

(179609, 31)

In [17]:
X = data.drop('Class',axis=1)
Y = data['Class']

In [32]:

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.05,random_state=42,stratify=Y)

In [35]:
X_test.to_csv("Test_credicard_data.csv",index=False)

In [33]:
X_test.shape

(8981, 30)

In [18]:
X.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
dtype: int64

In [20]:
model = IsolationForest(n_estimators=100, max_samples='auto', 
                                       contamination=0.2,
                                       random_state=42, 
                                       verbose=0)
model.fit(X)
print(model.get_params())
data['scores'] = model.decision_function(X)
data['anomaly_score'] = model.predict(X)
print('Anomalous Data')
print(data[data['anomaly_score'] == -1].head())



{'behaviour': 'deprecated', 'bootstrap': False, 'contamination': 0.2, 'max_features': 1.0, 'max_samples': 'auto', 'n_estimators': 100, 'n_jobs': None, 'random_state': 42, 'verbose': 0, 'warm_start': False}
Anomalous Data
    Time        V1        V2        V3  ...  Amount  Class    scores  anomaly_score
2    1.0 -1.358354 -1.340163  1.773209  ...  378.66    0.0 -0.042113             -1
7    7.0 -0.644269  1.417964  1.074380  ...   40.80    0.0 -0.067170             -1
14  12.0 -2.791855 -0.327771  1.641750  ...   58.80    0.0 -0.037651             -1
18  14.0 -5.401258 -5.450148  1.186305  ...   46.80    0.0 -0.094088             -1
21  17.0  0.962496  0.328461 -0.171479  ...   34.09    0.0 -0.030452             -1

[5 rows x 33 columns]


In [25]:
data[(data['Class'] ==1)&(data['anomaly_score'] == -1)].shape

(328, 33)

In [24]:
data[data['anomaly_score'] ==1].

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class,scores,anomaly_score
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0,0.050654,1
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0,0.051284,1
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0,0.005446,1
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0,0.027751,1
5,2.0,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,-0.371407,1.341262,0.359894,-0.358091,-0.137134,0.517617,0.401726,-0.058133,0.068653,-0.033194,0.084968,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.67,0.0,0.055252,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179601,124153.0,1.786904,-1.454071,-1.653961,-0.646983,-0.768945,-1.173636,0.014534,-0.384678,-0.333213,0.712413,-0.969361,-1.019346,-0.932178,0.232975,-0.114252,0.688785,0.461115,-1.735581,0.748139,0.333578,0.146130,-0.090930,-0.005531,-0.177174,-0.125407,-0.286648,-0.086505,-0.028427,237.20,0.0,0.012419,1
179602,124153.0,2.004683,-0.305812,-0.217588,0.326748,-0.708944,-0.670969,-0.502044,-0.092565,1.013671,-0.024845,-0.653864,0.695545,0.635286,-0.204531,0.523062,0.357872,-0.484407,-0.579498,-0.147507,-0.178328,-0.208897,-0.436137,0.429325,-0.004678,-0.633240,0.275635,-0.026685,-0.043308,1.00,0.0,0.053491,1
179603,124153.0,-0.155943,1.190031,-0.763020,-0.384402,0.839967,-1.109212,1.479676,-0.527897,-0.377126,0.090111,0.827006,0.880919,-0.065782,0.587057,-1.137250,-0.954758,-0.318527,0.025834,0.688493,-0.231398,0.373552,1.062604,-0.265388,0.135976,-0.168087,-0.201897,-0.442313,0.062345,18.13,0.0,0.020786,1
179604,124153.0,1.917694,-0.421875,-1.660221,0.724502,0.018688,0.103442,-0.618901,0.131122,-0.552475,0.253624,0.753776,-0.014623,-0.127352,-2.014848,-0.187300,-0.351763,1.008177,2.888547,-1.219172,-0.437165,-0.374483,-0.652282,0.152407,-0.000298,-0.174736,-0.699847,0.073080,0.021294,76.40,0.0,0.008371,1


## Model Evaluation

In [27]:
pred_anom = data[(data['Class'] == 1) & (data['anomaly_score'] ==-1)].shape[0]
total_anom = data[data['Class'] == 1].shape[0]
#model evaluation
accuracy = 100 * pred_anom /total_anom
print(f'accuracy of the model : {accuracy} %')

accuracy of the model : 90.10989010989012 %


## Save the model

In [28]:
import pickle
pickle_out = open("isolationforest.pkl","wb")
pickle.dump(model,pickle_out)
pickle_out.close()